In [2]:
from openai import AzureOpenAI
import os
import time


# Define state-saving functions
def save_state(state_file, last_processed_index):
    with open(state_file, 'w') as file:
        file.write(str(last_processed_index))

def load_state(state_file):
    try:
        with open(state_file, 'r') as file:
            return int(file.read().strip())
    except FileNotFoundError:
        return 0


daily_limit = 10000
rate_limit_per_minute = 500
state_file_path = 'last_processed_line.txt'


# Initialize the AzureOpenAI client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-01",
    azure_endpoint="https://llmsentimentanalysis.openai.azure.com/"
)

# Set the deployment name for your model
deployment_name = 'gpt35' 

def process_dataset_with_AzureOpenAI(df, column_name, state_file):
    start_index = load_state(state_file)  # Load the last processed index
    processed_count = 0

    for index, row in df.iterrows():
        if index < start_index:
            continue  # Skip already processed rows

        if processed_count >= daily_limit:
            print("Reached the daily limit, stopping...")
            break

        try:
            # Use the Azure client to send the prompt and get a response
            response = client.completions.create(
                model=deployment_name,
                prompt=row['Text'],
                max_tokens=10  # You may want to adjust max_tokens as needed
            )
            
            # Extract the generated text from the response
            generated_text = response.choices[0].text.strip()
            df.at[index, column_name] = generated_text

            # Save the state after each line is processed
            save_state(state_file, index)
            processed_count += 1

            # Azure may have different rate limiting, handle accordingly
            # if processed_count % rate_limit_per_minute == 0 and processed_count != 0:
            #     print("Rate limit reached, sleeping for 60 seconds...")
            #     time.sleep(60)
        except Exception as e:
            print(f"An error occurred at index {index}: {e}")
            continue  # Skip to next row if there was an error

    print(f"Processing completed. Data updated in the DataFrame.")
    return df


OpenAIError: Missing credentials. Please pass one of `api_key`, `azure_ad_token`, `azure_ad_token_provider`, or the `AZURE_OPENAI_API_KEY` or `AZURE_OPENAI_AD_TOKEN` environment variables.